In [ ]:
import torch
from torchvision.utils import make_grid
from matplotlib import pyplot as plt
from src.utils.dataset import IsingSampler
from src.utils.flow import GaussianConditionalProbabilityPath, LinearAlpha, LinearBeta, CFGVectorFieldODE, EulerSimulator
from architecture.nn_models import IsingNet
from src.utils.trainer import IsingTrainer
import os
import numpy as np

os.environ['CUDA_ALLOC_CONF'] = "expandable_segments:True"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Change these!
num_rows = 3
num_cols = 3
num_timesteps = 5

# Initialize our sampler
h5path = "../../data/gridstates_0.588_0.100.hdf5"
sampler = IsingSampler(h5path)

# Initialize probability path
path = GaussianConditionalProbabilityPath(
    p_data = sampler,
    p_simple_shape = [1, 64, 64],
    alpha = LinearAlpha(),
    beta = LinearBeta()
).to(device)

# Sample 
num_samples = num_rows * num_cols
z, _ = path.p_data.sample(num_samples)
z = z.view(-1, 1, 64, 64)

# Setup plot
fig, axes = plt.subplots(1, num_timesteps, figsize=(6 * num_cols * num_timesteps, 6 * num_rows))

# Sample from conditional probability paths and graph
ts = torch.linspace(0, 1, num_timesteps).to(device)
for tidx, t in enumerate(ts):
    tt = t.view(1,1,1,1).expand(num_samples, 1, 1, 1) # (num_samples, 1, 1, 1)
    xt = path.sample_conditional_path(z, tt) # (num_samples, 1, 64, 64)
    grid = make_grid(xt, nrow=num_cols, normalize=True, value_range=(-1,1), pad_value=1.0)
    axes[tidx].imshow(grid.permute(1, 2, 0).cpu(), cmap="gray")
    axes[tidx].axis("off")
plt.show()

In [ ]:
# Initialize probability path
path = GaussianConditionalProbabilityPath(
    p_data = sampler,
    p_simple_shape = [1, 64, 64],
    alpha = LinearAlpha(),
    beta = LinearBeta()
).to(device)

# Initialize model
unet = IsingNet(
    channels = [32, 64, 128],
    num_residual_layers = 2,
    t_embed_dim = 40,
    y_embed_dim = 40,
)

# Initialize trainer
trainer = IsingTrainer(path = path, model = unet, eta=0.1)

In [ ]:
# Train!
loss_history = trainer.train(num_epochs = 5000, device=device, lr=1e-3, batch_size=128)

In [ ]:
# Initialize probability path
path = GaussianConditionalProbabilityPath(
    p_data = sampler,
    p_simple_shape = [1, 64, 64],
    alpha = LinearAlpha(),
    beta = LinearBeta()
).to(device)

# Initialize model
unet = IsingNet(
    channels = [32, 64, 128],
    num_residual_layers = 2,
    t_embed_dim = 40,
    y_embed_dim = 40,
)

# Initialize trainer
trainer = IsingTrainer(path = path, model = unet, eta=0.1)

In [ ]:
torch.load("ising_model.pth", map_location=device)
unet.load_state_dict(torch.load("ising_model.pth", map_location=device))
unet.to(device)
unet.eval()

# Play with these!
samples_per_class = 5
num_timesteps = 100
guidance_scales = [1.0, 3.0, 5.0]

# Graph
fig, axes = plt.subplots(1, len(guidance_scales), figsize=(10 * len(guidance_scales), 10))

for idx, w in enumerate(guidance_scales):
    simulator = EulerSimulator(CFGVectorFieldODE(net=unet, guidance_scale=w))

    # Sample initial conditions
    y = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], dtype=torch.int64).repeat_interleave(samples_per_class).to(device)
    num_samples = y.shape[0]
    x0, _ = path.p_simple.sample(num_samples) # (num_samples, 1, 32, 32)

    # Simulate
    ts = torch.linspace(0,1,num_timesteps).view(1, -1, 1, 1, 1).expand(num_samples, -1, 1, 1, 1).to(device)
    x1 = simulator.simulate(x0, ts, y=y)

    # Plot
    grid = make_grid(x1, nrow=samples_per_class, normalize=True, value_range=(-1,1))
    axes[idx].imshow(grid.permute(1, 2, 0).cpu(), cmap="gray")
    axes[idx].axis("off")
    axes[idx].set_title(f"Guidance: $w={w:.1f}$", fontsize=25)
plt.show()

In [ ]:
ode = CFGVectorFieldODE(unet, guidance_scale=1.5)
simulator = EulerSimulator(ode)

num_samples = 500
x0, _ = path.p_simple.sample(num_samples)
ts = torch.linspace(0, 1, 100).view(1, -1, 1, 1, 1).expand(num_samples, -1, 1, 1, 1).to(device)
generated = simulator.simulate(x0.to(device), ts, y=torch.full((num_samples,), 20, dtype=torch.int64, device=device))
generated_values = generated.mean(dim=[1,2,3]).cpu()
real_values, _ = sampler.sample(num_samples)
real_values = real_values.mean(dim=[1,2,3]).cpu()

bins = np.linspace(-1, 1, 80)
plt.hist(real_values, bins=bins, alpha=0.6, label="Dataset", density=True)
plt.hist(generated_values, bins=bins, alpha=0.6, label="Unguided model", density=True)
plt.vlines(real_values.mean(), 0, plt.gca().get_ylim()[1], colors='k')
plt.legend()
plt.xlabel("Pixel value")
plt.ylabel("Density")
plt.title("Dataset vs. unguided model distribution")
plt.tight_layout()
plt.show()

In [ ]:
bins = np.linspace(-1, 1, 80)
plt.hist(real_values, bins=bins, alpha=0.6, label="Dataset", density=True)
plt.hist(generated_values, bins=bins, alpha=0.6, label="Unguided model", density=True)
#plt.vlines(real_values.mean(), 0, plt.gca().get_ylim()[1], colors='k')
plt.legend()
plt.xlabel("Magnetization")
plt.ylabel("Density")
plt.title("Dataset vs. unguided model distribution")
plt.tight_layout()
plt.show()